In [5]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler

In [6]:
def myknn(xt, xall, yall, k):
    dall = np.sum((xall - xt) ** 2, axis=1)
    orddall = np.argsort(dall)
    yhat = np.sign(np.sum(yall[orddall[:k]]))
    return yhat

In [7]:
# Carregar o conjunto de dados real (Iris Dataset)
iris = load_iris()
X = iris.data[:, :2]  # Usando apenas duas features para visualização
y = iris.target

# Convertendo para problema binário (-1 e 1)
y = np.where(y == 2, 1, -1)

# Normalizar os dados para melhor performance
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Parâmetros
k_values = range(1, 71)  # Variação de k
num_trials = 10  # Número de repetições para cálculo do desvio padrão (ajustado para testes rápidos)
np.random.seed(42)

# Listas para armazenar médias e desvios padrão de cada k
accuracies_mean = []
accuracies_std = []

In [ ]:
for trial in range(num_trials):
    accuracies_per_trial = []
    print(f"\nResultados do Experimento {trial+1}:")

    # Divisão entre treino e validação
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25, random_state=None)

    # Avaliação da acurácia para cada k
    for k in k_values:
        correct_predictions = 0
        for i in range(len(X_val)):
            prediction = myknn(X_val[i], X_train, y_train, k)
            if prediction == y_val[i]:
                correct_predictions += 1
        accuracy = correct_predictions / len(y_val)
        accuracies_per_trial.append(accuracy)

        # Gerando Fronteira de Decisão para cada k
        x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
        y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
        x_values = np.linspace(x_min, x_max, 100)
        y_values = np.linspace(y_min, y_max, 100)
        X_grid, Y_grid = np.meshgrid(x_values, y_values)

        # Classificando cada ponto da grade com k atual
        M = np.array([myknn(np.array([x, y]), X_train, y_train, k) for x, y in zip(X_grid.ravel(), Y_grid.ravel())])
        M = M.reshape(X_grid.shape)

        # Plotar a Fronteira de Decisão
        plt.figure(figsize=(8, 6))
        plt.contourf(X_grid, Y_grid, M, cmap='coolwarm', alpha=0.7)
        plt.scatter(X_train[:, 0], X_train[:, 1], c=y_train, cmap='coolwarm', edgecolors='black')
        plt.xlabel('Feature 1')
        plt.ylabel('Feature 2')
        plt.title(f'Fronteira de Decisão - Experimento {trial+1}, k={k}')
        plt.show()

    # Gráfico de Acurácia por Experimento
    plt.figure(figsize=(8, 6))
    plt.plot(k_values, accuracies_per_trial, 'o-', label=f'Experimento {trial+1}')
    plt.xlabel('Número de Vizinhos (k)')
    plt.ylabel('Acurácia')
    plt.grid()
    plt.legend()
    plt.show()

In [ ]:
# Cálculo da Média e Desvio Padrão
print("\nResultados Finais:")
for k in k_values:
    trial_accuracies = []
    for _ in range(num_trials):
        X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25, random_state=None)
        correct_predictions = 0
        for i in range(len(X_val)):
            prediction = myknn(X_val[i], X_train, y_train, k)
            if prediction == y_val[i]:
                correct_predictions += 1
        accuracy = correct_predictions / len(y_val)
        trial_accuracies.append(accuracy)

    mean_accuracy = np.mean(trial_accuracies)
    std_dev_accuracy = np.std(trial_accuracies)
    accuracies_mean.append(mean_accuracy)
    accuracies_std.append(std_dev_accuracy)
    print(f"k={k}: Média = {mean_accuracy:.4f}, Desvio Padrão = {std_dev_accuracy:.4f}")

# **Gráfico Final: Acurácia Média e Desvio Padrão**
plt.figure(figsize=(12, 6))
plt.errorbar(k_values, accuracies_mean, yerr=accuracies_std, fmt='o-', label='Acurácia Média', capsize=5)
plt.xlabel('Número de Vizinhos (k)')
plt.ylabel('Acurácia Média')
plt.grid()
plt.legend()
plt.show()